Import parsl

In [4]:
import parsl
from parsl.config import Config
from parsl.executors.threads import ThreadPoolExecutor
from parsl.app.app import bash_app,python_app
from parsl import File
#import sys

Configure parsl

In [5]:
config = Config(
    executors=[ThreadPoolExecutor()],
    lazy_errors=True
)
parsl.load(config)

Define parsl Apps that call each of the data transformation scripts we'll use

In [13]:
## Define Apps ##
@bash_app
def WireDelay(threshIn='', outputs=[], geoDir='', daqId='', fw='', stdout='stdout.txt', stderr='stderr.txt'):
    return 'perl ./perl/WireDelay.pl %s %s %s %s %s' %(threshIn, outputs[0], geoDir, daqId, fw)

@bash_app
def Combine(inputs=[], outputs=[], stdout='stdout.txt', stderr='stderr.txt'):
    return 'perl ./perl/Combine.pl ' + ' '.join(inputs) + ' ' + str(outputs[0])

@bash_app
def SingleChannel(inputs=[], outputs=[], channel='1'):
    return 'perl ./perl/SingleChannel.pl %s %s %s' % (inputs[0], outputs[0], channel)
    
@bash_app
def Sort(inputs=[], outputs=[], key1='1', key2='1', stdout='stdout.txt', stderr='stderr.txt'):
    return 'perl ./perl/Sort.pl %s %s %s %s' %(inputs[0], outputs[0], key1, key2)

@bash_app
def Flux(inputs=[], outputs=[], binWidth='600', geoDir='geo/', stdout='stdout.txt', stderr='stderr.txt'):
    return 'perl ./perl/Flux.pl %s %s %s %s' %(inputs[0], outputs[0], binWidth, geoDir)

In [14]:
## Analysis Parameters ##
# Define input parameters

# For WireDelay
thresholdAll = ('files/6119.2016.0104.1.thresh', 'files/6203.2016.0104.1.thresh')
wireDelayData = ('6119.2016.0104.1.wd', '6203.2016.0104.1.wd')
geoDir = './geo'
detectors = ('6119', '6203')
firmwares = ('1.12', '1.12')

# For Combine
combineOut = 'combineOut'

# For SingleChannel
singlechannel_channel = '1'
singleChannelOut = 'singleChannelOut'

# For Sort
sort_sortKey1 = '2'
sort_sortKey2 = '3'
sortOut = 'sortOut'

# For Flux
binWidth = '600'
geoFiles = ['geo/6119/6119.geo', 'geo/6203/6203.geo']
fluxOut = 'fluxOut'

In [15]:
## Workflow ##
# 1) WireDelay() takes input Threshold (.thresh) files and converts
#    each to a Wire Delay (.wd) file:
WireDelay_futures = []
for i in range(len(thresholdAll)):
        WireDelay_futures.append(WireDelay(threshIn=thresholdAll[i], outputs=[wireDelayData[i]], geoDir=geoDir, daqId=detectors[i], fw=firmwares[i]))

# WireDelay_futures is a list of futures.
# Each future has an outputs list with one output.
WireDelay_outputs = [i.outputs[0] for i in WireDelay_futures]

# 2) Combine() takes the WireDelay files output by WireDelay() and combines
#    them into a single file with name given by `combineOut`
Combine_future = Combine(inputs=WireDelay_outputs, outputs=[combineOut])

# 3) SingleChannel() takes
SingleChannel_future = SingleChannel(inputs=Combine_future.outputs, outputs=[singleChannelOut], channel=singlechannel_channel)

# 4) Sort() sorts the `singleChannelOut` file, producing a new file with name given
#    by `sortOut`
Sort_future = Sort(inputs=SingleChannel_future.outputs, outputs=[sortOut], key1=sort_sortKey1, key2=sort_sortKey2)

# 5) Flux() processes the `sortOut` file and produces a output file with name given 
#    by `fluxOut`
Flux_future = Flux(inputs=Sort_future.outputs, outputs=[fluxOut], binWidth=binWidth, geoDir=geoDir)

# Wait for the final result before exiting.
x = Flux_future.result()

print("Call to Flux completed with exit code:", x)

Call to Flux completed with exit code: 0
